In [1]:
# Reading/Writing Data
import os
import h5py as hp
import numpy as np
import urllib
import math
import matplotlib.pyplot as plt

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F


from torch.utils.tensorboard import SummaryWriter

In [2]:
electron = "Electron.hdf5"
photon  = "Photon.hdf5"

electron_url = 'https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
photon_url = 'https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'

In [3]:
def download(url, filename):

    if filename not in os.listdir():
        urllib.request.urlretrieve(url, filename)

download(electron_url, electron)
download(photon_url,   photon)

In [4]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(length, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * length)
    train_set_size = length - valid_set_size
    index = range(length)
    train_ind, valid_ind = random_split(index, \
    [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_ind), np.array(valid_ind)

def data_read(pa1, pa2):
    f1 = hp.File(pa1)
    x1 = np.array(f1['X'])
    y1 = np.array(f1['y'])

    f2 = hp.File(pa2)
    y2 = np.array(f2['y'])
    x2 = np.array(f2['X'])
    
    X = np.concatenate([x1, x2], axis=0)
    y = np.concatenate([y1, y2], axis=0, dtype=np.float32)
    # X = X
    length = X.shape[0]

    return X, y, length



X, y, length = data_read(electron, photon)
print(X.shape, y.shape, length)


(498000, 32, 32, 2) (498000,) 498000


In [5]:

foo = X[:3]
foo = torch.Tensor(foo)
print(f'foo = {foo.shape}')

t = nn.Conv2d(32, 12, 1)
t(foo).shape


foo = torch.Size([3, 32, 32, 2])


torch.Size([3, 12, 32, 2])

In [6]:
print(X[1].shape)

(32, 32, 2)


## `EPData` class

In [23]:
# Dataset

class EPData(Dataset):
    def __init__(self, X, y=None):
        super().__init__()

        self.X = X
        self.y = y
    
    def __getitem__(self, ind):
        if self.y is None:
            return torch.Tensor([self.X[ind][:, :, 0], self.X[ind][:, :, 1]])
        else:
            return torch.Tensor([self.X[ind][:, :, 0], self.X[ind][:, :, 1]]), torch.tensor(self.y[ind])

    def __len__(self):
        return len(self.X)

## `Model` class

In [24]:
# model

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, bias=False, padding=0)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut.add_module('conv', nn.Conv2d(in_channels, out_channels, 3, bias=False, padding=0))
            self.shortcut.add_module('bn', nn.BatchNorm2d(out_channels))
        
    
    def forward(self, x):
        y = F.relu(self.bn1(self.conv1(x)), inplace=True)
        y = F.relu(self.bn2(self.conv2(y)), inplace=True)
        y = y + self.shortcut(x)
        # y = F.relu(y, inplace=True)
        return y


class EPNet(nn.Module):
    def __init__(self, config):
        super(EPNet, self).__init__()
        shape = config['input_shape']
        input_channel = config['input_channels']
        self.stage1 = BasicBlock(input_channel, 32)
        self.stage2 = BasicBlock(32, 64)
        self.stage3 = BasicBlock(64, 32)
        with torch.no_grad():
            self.feature = self._forward_test(torch.zeros(shape)).view(-1).shape[0]
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.feature, config['n_classes']),
            nn.Sigmoid()
        )
    
    def _forward_test(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        # x = self.stage3(x)
        x = F.adaptive_avg_pool2d(x, output_size=1)
        # print("average pool:", x.shape)
        return x

    def forward(self, x):
        x = self._forward_test(x)
        # x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


class test(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Flatten(1),
            nn.Sigmoid(),
            nn.Linear(2048, 1)
        )

    def forward(self, x):
        return self.layer(x)



In [25]:
# model

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False)
# model = test()
# model(foo).shape

## configuration

#### `mps` on mac, `cuda` on pc and the `parameter` of the model

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# configration of the model
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 300,     # Number of epochs.
    'n_classes': 1,
    'base_channels': 3,
    'input_channels': 2,
    'input_shape': (1, 2, 32, 32),
    'depth': 4,
    'block_type': 'basic',
    'batch_size': 2048,
    'learning_rate': 1e-3,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}
print(config)

{'seed': 5201314, 'select_all': True, 'valid_ratio': 0.2, 'n_epochs': 300, 'n_classes': 1, 'base_channels': 3, 'input_channels': 2, 'input_shape': (1, 2, 32, 32), 'depth': 4, 'block_type': 'basic', 'batch_size': 2048, 'learning_rate': 0.001, 'early_stop': 400, 'save_path': './models/model.ckpt'}


In [27]:
# print(length)
train_ind, valid_ind = train_valid_split(length, config['valid_ratio'], config['seed'])

x_train, y_train, x_valid, y_valid = X[train_ind], y[train_ind], X[valid_ind], y[valid_ind]

train_dataset, valid_dataset = EPData(x_train, y_train), \
                               EPData(x_valid, y_valid)


# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)

# test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)





In [28]:
model = EPNet(config)
# count = 0
# for x, y in train_loader:
#     count += 1
#     if count <= 50:
#         print(x.shape)
print(train_dataset[0][0].shape)

torch.Size([2, 32, 32])


## Train

In [29]:
%matplotlib notebook

In [30]:
# # model = test()
# train_loss_record = []
# valid_loss_record = []
# train_steps = []
# valid_steps = []

# def train(model, train_loader, valid_loader, config, device):

#     model.to(device)

#     # optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.85, weight_decay=0.8)

#     optimizer = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-5)
#     # ll = nn.MSELoss()
#     ll = nn.CrossEntropyLoss()

#     if not os.path.isdir('./models'):
#         os.mkdir('./models')

#     n_epochs, best_loss, step, valid_step = config['n_epochs'], -math.inf, 0, 0

#     fig = plt.figure()
#     ax = fig.add_subplot(211)
#     ax.set_title("train")
#     bx = fig.add_subplot(212)
#     bx.set_title("valid")
#     fig.show()

#     for epoch in range(n_epochs):
#         model.train()

#         for x, y in train_loader:
#             loss_record = []
#             x = x.to(device)
#             y = y.type(torch.float32)
#             y = y.to(device)
#             optimizer.zero_grad()
#             predic = model(x)
#             loss = ll(predic, y)
#             loss.backward()
#             optimizer.step()

#             loss_record.append(loss.detach().item())
#             # print(loss)

#         train_loss_record.append(sum(loss_record)/len(loss_record))
#         train_steps.append(epoch)
#         ax.plot(train_steps, train_loss_record)
#         fig.canvas.draw()
        
#         for x, y in valid_loader:
#             loss_record = []
#             x, y = x.to(device), y.type(torch.float32).to(device)
#             with torch.no_grad():
#                 model.eval()
#                 pre = model(x)
#                 loss = ll(pre, y)
#             loss_record.append(loss.detach().item())
            
#         valid_loss_record.append(sum(loss_record)/len(loss_record)) 
#         valid_steps.append(epoch)
#         bx.plot(valid_steps, valid_loss_record)
#         fig.canvas.draw()
        
            

# train(model, train_loader, valid_loader, config, device)

In [31]:
# Train
# model = test()
def train(model, train_loader, Valid_loader, config, device):
    # ll = nn.MSELoss()
    ll = nn.CrossEntropyLoss()

    # optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.8)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=1e-5)
    
    writer = SummaryWriter() # Writer of tensoboard.
    if not os.path.isdir('./models'):
        os.mkdir('./models')
    
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        model.to(device)
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            # print(x.dtype, y.dtype)
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), ytype(torch.LongTensor)   .to(device)   # Move your data to device.
            pred = model(x)
            loss = ll(pred, y)
            # print(loss.dtype)
            
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)

        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                # print(f'pred.shape = {pred.shape} y.shape = {y.shape}')
                loss = ll(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

train(model, train_loader, valid_loader, config, device)


  0%|          | 0/195 [00:00<?, ?it/s]


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

In [ ]:
# valid
mol = torch.load("./model.ckpt")

model_valid = EPNet(config)
model_valid.load_state_dict(mol)
model_valid.to(device)
right = 0

n = 0

for x, y in valid_loader:
    # mol.eval()
    # mol.to('cpu')
    x, y = x.to(device), y.to(device)
    n += 1

    
    with torch.no_grad():
        predict = model_valid(x)
        # torch.argmax(predict[i]) == torch.argmax(y):
        #     right += 1
        right += torch.sum(predict, dim=1 == y, dim=1)
        n += config['batch_size']

print(right / n)

In [ ]:
# valid
mol = torch.load("./model.ckpt")
model_valid = EPNet(config)
model_valid.load_state_dict(mol)
right = 0

n = 0

for x, y in valid_loader:
    mol.valid()
    mol.to('cpu')
    n += 1
    with torch.no_grad():
        predict = mol(x)
        predict = torch.argmax(predict)
        right += torch.sum(predict, dim=1 == y, dim=1)
        n += config['batch_size']
print(right / n)